In [1]:
import pandas.io.sql as psql
import re
import psycopg2
from gensim.models.word2vec import Word2Vec

In [2]:
import json
with open('/Users/krishna/MOOC/smart-city/login.json') as data_file:
    db = json.load(data_file)

conn = psycopg2.connect(database=db['dbname'], user=db['user'], password=db['password'], host=db['host'], port=db['port'])

In [3]:
dataframe = psql.frame_query("select text from twitter.tweets limit 1000000",conn)

/usr/local/lib/python2.7/site-packages/pandas/io/sql.py:1569: FutureWarning: frame_query is deprecated, use read_sql
  warnings.warn("frame_query is deprecated, use read_sql", FutureWarning)


In [4]:
dataframe.head()

,text
0,RT @jersey_ciamikk: #jersey4sale #1308 Lyon Te...
1,RT @EmpireFOX: ⚡️ #Empire LIVE: The Lyon famil...
2,"RT @WeNeedTrump: ""Journalists are routinely pa..."
3,The Lyon are back https://t.co/FuJzPGbYKl
4,RT @EmpireFOX: Lyon Dynasty is our company. Bu...


In [5]:
def processTweet(tweet):
    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+',' ',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    # Remove some punctuations
    tweet = re.sub(r'[-:.,]', '', tweet)
    
    #trim
    tweet = tweet.strip('\'"')
    return tweet.split(" ")
x_train = (dataframe['text']).apply(lambda x : processTweet(x))
x_train.head()

0    [rt, jersey4sale, 1308, lyon, techfit, sz, 6, ...
1    [rt, ⚡️, empire, live, the, lyon, family, is, ...
2    [rt, "journalists, are, routinely, paid, by, t...
3                          [the, lyon, are, back, URL]
4    [rt, lyon, dynasty, is, our, company, but, emp...
Name: text, dtype: object

In [6]:
min_count = 1
size = 50
window = 5
model = Word2Vec(x_train, min_count=min_count, size=size, window=window)

In [7]:
model.most_similar("awesome")

[('using', 0.8294156193733215),
 ('amazing', 0.8200231194496155),
 ('poor', 0.785050630569458),
 ('official', 0.7690879702568054),
 ('system', 0.7619841694831848),
 ('fucking', 0.7587185502052307),
 ('healing', 0.7496306300163269),
 ('funny', 0.7469406127929688),
 ('fine', 0.7444379329681396),
 ('awful', 0.7427728176116943)]

In [8]:
model.similarity("paris","lyon")

0.54171155109306535

In [9]:
model['lyon']

array([ 0.86924994, -4.9789834 ,  1.3708179 ,  0.9778772 , -0.02370777,
        0.76255035, -4.09478331, -1.13887632,  2.13752627,  2.04903221,
        1.32424414, -0.81374109, -0.94304085, -2.38849044,  1.02972889,
       -2.55333042, -0.12142814,  0.83060884, -4.31650639, -0.42551121,
       -0.93681282,  2.73746467,  0.36426023,  3.67662048, -0.70349705,
        1.97618496,  3.20900965, -1.96379614, -3.3901782 ,  2.0519805 ,
       -2.83937216,  4.37276363, -0.92251778,  0.04471493, -1.22538817,
       -0.12857522,  1.12794185,  1.13678133,  2.79417324, -2.59652901,
        0.95113552,  1.37113798,  3.67619181, -0.67609823, -1.56411529,
        0.77392948, -1.0804522 , -0.4784795 , -2.51273537,  0.47444874], dtype=float32)

In [10]:
model.most_similar(positive=['lyon'], negative=['film'])

[('idn', 0.5748438835144043),
 ('voil\xc3\xa0"', 0.5748145580291748),
 ('atless', 0.5410152673721313),
 ('l\xe2\x80\x99aff\xc3\xbbt', 0.5288395285606384),
 ('duch&amp;egrave;re', 0.5260607004165649),
 ('barcelone', 0.525658130645752),
 ('(3\xc2\xba', 0.5250025391578674),
 ('h/sem', 0.5186048150062561),
 ("(62'", 0.5147116780281067),
 ('caen', 0.5097489953041077)]